Импортируем нужные библиотеки.

In [2]:
import numpy as np 
import pandas as pd 
import fasttext
from faker import Faker
from sdv.tabular import GaussianCopula

Для генерации фамилий датасета решено использовать библиотеку, которая генерирует синтетические данные. 

Конкретно здесь используются метод last_name, который сгенерирует массив из 100 тысяч фамилий, позже преобразованный в датафрейм.

In [3]:
fake = Faker("en_US")

In [4]:
surnames = []
for _ in range(100000):
    surnames.append(fake.last_name())

In [5]:
df = pd.DataFrame(surnames)
df.head()

,0
0,Hicks
1,Reed
2,Hudson
3,Garcia
4,Adams


Нулевых значений в таблице нет (но если бы были, их нужно было бы убрать).

In [6]:
df=df.dropna(axis=0)

Меняем название столбца для удобства.

In [7]:
df.columns = ['last_name']

In [8]:
df.head()

,last_name
0,Hicks
1,Reed
2,Hudson
3,Garcia
4,Adams


Поскольку в сгенерированном датасете достаточно много повторений, было также принято решение дополнить датасет 
данными, сгенерированными на основе первого датасета. 

Для этого используется библиотека SDV, с помощью которого добавляем в датасет 50 тысяч новых значений.

Соединяем датасеты.

In [9]:
model = GaussianCopula()
model.fit(df)

In [10]:
sample = model.sample(50000)
sample.head()

,last_name
0,Wood
1,Green
2,Stone
3,Ferguson
4,Carter


In [11]:
df = pd.concat([df,sample])

Посмотрим на дупликаты. Их много...

In [12]:
dups = df.pivot_table(columns=['last_name'], aggfunc='size')
print(dups)

last_name
Abbott        44
Acevedo       45
Acosta        93
Adams        614
Adkins       126
            ... 
Zamora        52
Zavala        40
Zhang         37
Zimmerman    145
Zuniga        47
Length: 1000, dtype: int64


Поскольку данные представлены в "чистом" виде, предварительная обработка (с помощью регулярных выражений, например) не нужна. 

Предварительная обработка нужна для предложений, больших текстов.

In [13]:
def preprocess(text):
    #text = re.sub(r'[^\w\s\']',' ', text)
    #text = re.sub(r'[ \n]+', ' ', text)
    #text = re.sub(r'[0-9]+', ' ', text)
    return text.strip().lower() 

In [14]:
df.last_name = df.last_name.map(preprocess)

In [15]:
df.to_csv("surnames.txt", columns=["last_name"], header=None, index=False)

Самая главная часть – создание модели на основе библиотеки FastText. 

FastText специализируется на создании векторов слов, которые помогают находить семантическую похожесть или
похожесть по составу.

Наиболее важными параметрами модели являются ее размерность и диапазон размеров подслов. Размерность (dim) 
определяет размер векторов, чем они больше, тем больше информации они могут захватить, но для изучения требуется 
больше данных. Но, если они слишком большие, их труднее и медленнее тренировать. По умолчанию мы используем 
100 размеров, но также популярны любые значения от 100 до 300. Подслова — это все подстроки, содержащиеся 
в слове между минимальным размером (minn) и максимальным размером (maxn). По умолчанию мы берем все подслова 
от 3 до 6 символов, но для разных языков может быть более подходящим другой диапазон.

In [17]:
model = fasttext.train_unsupervised("surnames.txt", minn=3, maxn=6, dim=300)

Read 0M words
Number of words:  1001
Number of labels: 0
Progress: 100.0% words/sec/thread: 2030119 lr:  0.000000 avg.loss:  3.514271 ETA:   0h 0m 0s


У FastText есть несколько методов, которые могут найти похожие значения из словаря. 

Например, get_nearest_neighbors показывает похожие слова. Но в данном случае поиск семантически похожих слов не имеет смысла, поскольку мы работаем с фамилиями.

In [18]:
model.get_nearest_neighbors("rich")

[(0.9996427297592163, 'richard'),
 (0.9996366500854492, 'richards'),
 (0.999620795249939, 'richardson'),
 (0.9996066689491272, 'harrington'),
 (0.99959397315979, 'rice'),
 (0.9995939135551453, 'roman'),
 (0.9995881915092468, 'harrell'),
 (0.9995880722999573, 'farrell'),
 (0.999586820602417, 'hartman'),
 (0.9995819926261902, 'rivers')]

In [19]:
model.get_nearest_neighbors("ander")

[(0.9998788237571716, 'morrison'),
 (0.9998705983161926, 'alexander'),
 (0.9998663067817688, 'harrell'),
 (0.9998652338981628, 'bender'),
 (0.9998621940612793, 'richardson'),
 (0.9998568296432495, 'andersen'),
 (0.999854326248169, 'harrison'),
 (0.9998538494110107, 'sanders'),
 (0.9998524188995361, 'holder'),
 (0.9998509287834167, 'garrison')]

Какой итог можно сделать?

Как мне показалось, модель не совсем хорошо справляется с поиском слов, похожих по составу. Возможно, дело не в самом хорошем сгенерированном датасете.